In [ ]:
from paths import get_path_urlsinfo

In [ ]:
PATH_URLSINFO=get_path_urlsinfo()

In [ ]:
PATHNAME_INPUT=PATH_URLSINFO+'urls_info.csv'
PATHNAME_OUTPUT=PATH_URLSINFO+'urls_info_and_domain.csv'

In [ ]:
import pandas as pd

In [ ]:
from utils_domain import  get_url_domain

In [ ]:
URLS_INFO=pd.read_csv(PATHNAME_INPUT)

In [ ]:
URLS_INFO.status_code.value_counts(normalize=True)*100

In [ ]:
URLS_INFO[URLS_INFO.status_code=='FAILED'].reason.value_counts()/len(URLS_INFO)*100

In [ ]:
URLS_INFO[URLS_INFO.reason=='UnknownError'].url.values[:30]

# Domain extraction

In [ ]:
def get_domain(url):
    if get_url_domain(url)=='web.archive.org':
        return get_url_domain(url[url.rfind('http'):])
    return get_url_domain(url)

In [ ]:
URLS_INFO['initial_domain'] =URLS_INFO['url'].map(get_domain)
URLS_INFO['redirect_domain']=URLS_INFO['destination_url'].map(get_domain)

#### Extract best domain

In [ ]:
def get_final_domain(row):
    # failure of initial url
#     if row.reason=='InvalidUrl':
#         return 'Invalid'
    # failure of redirect
    if row.redirect_domain=='FAILED':
        return row.initial_domain
    # everything working
    return row.redirect_domain

In [ ]:
URLS_INFO['final_domain']=URLS_INFO.apply(get_final_domain, axis=1)

In [ ]:
# URLS_INFO=URLS_INFO[URLS_INFO.status_code!='FAILED']

In [ ]:
URLS_INFO['final_domain'].value_counts()[:30]

In [ ]:
URLS_INFO[URLS_INFO['final_domain']=='FAILED']

In [ ]:
URLS_INFO.final_domain.value_counts()[:30]

In [ ]:
URLS_INFO[URLS_INFO.redirect_domain!=URLS_INFO.initial_domain]

# Merge with perennial sources info

In [ ]:
PERENNIAL_INFO=pd.read_csv(f'{PATH_URLSINFO}enwiki_perennial_30_03_2023.csv')

In [ ]:
PERENNIAL_INFO#[50:100]

In [ ]:
# PERENNIAL_SAMPLE_URLS=PERENNIAL_INFO.sample(5).url.tolist()
PERENNIAL_SAMPLE_URLS=['nationalgeographic.com', 'bloomberg.com', 'theguardian.com', 'vogue.com', 
                       'mirror.co.uk', 'cosmopolitan.com', 'chinadaily.com.cn',
                       'facebook.com', 'youtube.com', 'arxiv.org', 'researchgate.net']
PERENNIAL_SAMPLE=PERENNIAL_INFO[PERENNIAL_INFO.url.isin(PERENNIAL_SAMPLE_URLS)]\
.drop(columns=['last']).drop_duplicates(subset='url')\
.set_index('url')
PERENNIAL_SAMPLE=PERENNIAL_SAMPLE.loc[PERENNIAL_SAMPLE_URLS]

In [ ]:
# Function to apply a background color
def colorize(row):
    if row['status'] == 'Generally unreliable':
        return ['background-color: red'] * (len(row))
    if row['status'] == 'No consensus':
        return ['background-color: yellow'] * (len(row))
    if row['status'] == 'Generally reliable':
        return ['background-color: green'] * (len(row))
    return [''] * len(row)

# Apply the coloring function
styled_df = PERENNIAL_SAMPLE.style.apply(colorize, axis=1)
styled_df

In [ ]:
styled_df

In [ ]:
len(PERENNIAL_INFO[PERENNIAL_INFO.url.isin(URLS_INFO.final_domain)])

#### Statistics across perennial sources used (above) vs not used (blow) in dataset

In [ ]:
PERENNIAL_INFO[PERENNIAL_INFO.url.isin(URLS_INFO.final_domain)].status.value_counts(normalize=True)*100

In [ ]:
PERENNIAL_INFO[~PERENNIAL_INFO.url.isin(URLS_INFO.final_domain)].status.value_counts(normalize=True)*100

In [ ]:
URLS_INFO

In [ ]:
PERENNIAL_INFO[['url', 'status']].set_index('url').rename(columns={'status':'perennial_status'})

In [ ]:
PERENNIAL_INFO['status'].value_counts()

In [ ]:
PERENNIAL_INFO[PERENNIAL_INFO['status']=='Blacklisted'][30:60]

In [ ]:
URLS_INFO=URLS_INFO.merge(PERENNIAL_INFO[['url', 'status']].set_index('url').rename(columns={'status':'perennial_status'}), 
                          left_on='final_domain', right_index=True, suffixes=('','_perennial'), how='left')

In [ ]:
URLS_INFO['final_domain']=URLS_INFO['final_domain'].fillna('FAILED')
URLS_INFO['final_domain']=URLS_INFO['final_domain'].map(lambda x: 'FAILED' if x=='' else x)

In [ ]:
URLS_INFO['perennial_status']=URLS_INFO['perennial_status'].fillna('unknown')

In [ ]:
URLS_INFO.perennial_status.value_counts(normalize=True)*100

# Merge with MBFC data

In [ ]:
# Data from https://github.com/drmikecrowe/mbfcext/blob/main/docs/v5/data/sources.json

In [ ]:
MBFC_DATA=pd.read_json(f'{PATH_URLSINFO}mbtf_sources_2.json')
len(MBFC_DATA)

In [ ]:
MBFC_DATA

#### Domain compatible with ours

In [ ]:
MBFC_DATA['clean_domain']=MBFC_DATA['domain'].map(get_domain)
# MBFC_DATA[MBFC_DATA['clean_domain']!=MBFC_DATA['domain']]

#### Removing non entries (reporting=nan)

In [ ]:
MBFC_DATA.credibility.unique()

In [ ]:
# MBFC_DATA[MBFC_DATA.reporting.map(lambda x: not isinstance(x,str))]

In [ ]:
MBFC_DATA=MBFC_DATA.dropna(subset=['reporting'])

#### Statistics

In [ ]:
MBFC_DATA.reporting.value_counts(normalize=True)*100

In [ ]:
MBFC_DATA[MBFC_DATA['clean_domain'].isin(URLS_INFO['final_domain'])].reporting.value_counts(normalize=True)*100

#### Cleaning duplicated (same domain different sections)

In [ ]:
# drop all duplicated if they have same reporting
MBFC_DATA=MBFC_DATA.drop_duplicates(subset=['clean_domain', 'reporting'])

In [ ]:
MBFC_DATA[MBFC_DATA['clean_domain']=='huffpost.com']

In [ ]:
REMAINING_DUPLICATES=MBFC_DATA[MBFC_DATA['clean_domain'].duplicated(keep=False)]
TO_REMOVE_IDX=REMAINING_DUPLICATES[~REMAINING_DUPLICATES.domain.isin(URLS_INFO.final_domain)].index
MBFC_DATA.loc[TO_REMOVE_IDX]

In [ ]:
MBFC_DATA=MBFC_DATA[~MBFC_DATA.index.isin(TO_REMOVE_IDX)]

In [ ]:
MBFC_DATA.clean_domain.value_counts()

### Merging

In [ ]:
len(URLS_INFO)

In [ ]:
URLS_INFO=URLS_INFO.merge(MBFC_DATA[['clean_domain', 'reporting']].set_index('clean_domain').rename(columns={'reporting':'mbfc_status'}),
                          left_on='final_domain', right_index=True, how='left')

In [ ]:
URLS_INFO['mbfc_status']=URLS_INFO['mbfc_status'].fillna('unknown')

# Saving data

In [ ]:
PATHNAME_OUTPUT

In [ ]:
URLS_INFO=URLS_INFO.drop_duplicates(subset='url')

In [ ]:
URLS_INFO[URLS_INFO.url=='https:///books?id=GtBa6XIW_aQC&pg=PA77'].final_domain.values

In [ ]:
URLS_INFO.to_csv(PATHNAME_OUTPUT, index=False)

# Statistics

In [ ]:
len(URLS_INFO.final_domain.unique())

In [ ]:
len(URLS_INFO)

In [ ]:
PERENNIAL_INFO[PERENNIAL_INFO.url.isin(URLS_INFO.final_domain)]

In [ ]:
PERENNIAL_INFO[~PERENNIAL_INFO.url.isin(URLS_INFO.final_domain)]

In [ ]:
len(PERENNIAL_INFO[~PERENNIAL_INFO.url.isin(URLS_INFO.final_domain)].name.unique())

In [ ]:
PERENNIAL_INFO[PERENNIAL_INFO.url.map(lambda x: '/' in x)]

In [ ]:
PERENNIAL_INFO.drop_duplicates(subset=['url'])

In [ ]:
PERENNIAL_INFO[:10]

In [ ]:
URLS_INFO['perennial_status'].value_counts(normalize=True)*100

In [ ]:
URLS_INFO['mbfc_status'].value_counts(normalize=True)*100

In [ ]:
URLS_INFO['final_domain'].value_counts(normalize=True)*100